<a href="https://colab.research.google.com/github/mjgroth/deephys-aio/blob/master/Python_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1. Extract Neural Activity (example with Torch)

Desired output:

    [all_activs,all_outputs], #Lists with neural activity
    all_images, #Images resized to 32x32 pixels
    all_cats #Labels

In [ ]:
!pip install torch torchvision

from __future__ import print_function
import argparse
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable


In [3]:
#Argument initializatio
args={}
kwargs={}
args['batch_size']=128
args['cuda']=True #Enabling cuda is required

In [4]:
#Neural network model definition (Different ResNet versions)
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear1 = nn.Linear(512*block.expansion, 50)
        self.linear2 = nn.Linear(50, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear1(out)
        out = self.linear2(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


Please download [this Google Drive file](https://drive.google.com/file/d/1oUMHbE9ck1Wgi_C-RwmnP_w64IFdL3uK/view?usp=sharing) (`cifar_resnet18.pth`) and [this one](https://drive.google.com/file/d/1VtgCfn5YqYs1zJPyj6b5suv_xHlpJ_K1/view?usp=sharing) (`cifar102_train.npz`) too to continue

Or use PyDrive below (tutorial reference [link text](https://sigmundojr.medium.com/how-do-i-read-a-csv-file-from-google-drive-using-python-colab-966091922852))

In [5]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileDownloaded = drive.CreateFile({"id":"1oUMHbE9ck1Wgi_C-RwmnP_w64IFdL3uK"})
fileDownloaded.GetContentFile("cifar_resnet18.pth")
fileDownloaded = drive.CreateFile({"id":"1VtgCfn5YqYs1zJPyj6b5suv_xHlpJ_K1"})
fileDownloaded.GetContentFile("cifar102_train.npz")

In [6]:
#Save the activations for penultimate layer

#Select a model
models = ResNet18()
if args['cuda']:
    models.cuda()

#Load the saved model
state_dict = torch.load('cifar_resnet18.pth')
models.load_state_dict(state_dict)

#Function to extract activations and the logits
from collections import defaultdict
def act_extract(testloader, model):
    all_activs = []
    all_outputs = []
    all_images = []
    all_cats = []
    activation = {}
    #Standard normalization for CIFAR10 versions
    normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    # resize = transforms.Resize((32,32)) #default size is 32 for deephys
    a_s = defaultdict(list)
    iterator = iter(testloader)
    
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    
    for i in range(len(testloader)):
        # print (i)
        data, target = next(iterator)

        all_images.append(data)
        all_cats.append(target)

        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data = normalize(data) #Required to capture activations with normalized data

        model.linear1.register_forward_hook(get_activation('linear1'))
        with torch.no_grad():
            activation['output'] = model(data).detach().cpu()#.numpy()
            all_outputs.append(activation['output'])
            activation['linear1'] = F.relu(activation['linear1']).detach().cpu()#.numpy()
            all_activs.append(activation['linear1'])
            
    all_activs = torch.cat(all_activs)
    all_outputs = torch.cat(all_outputs)
    all_images = torch.cat(all_images)
    all_cats = torch.cat(all_cats).numpy().tolist()
    
    return all_activs, all_outputs, all_images, all_cats

In [ ]:
transform_test = transforms.Compose([
    transforms.ToTensor()
])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(testset, batch_size=args['batch_size'],
                                         shuffle=False, num_workers=2)

all_activs, all_outputs, all_images, all_cats = act_extract(testloader, models)

In [8]:
#Collect neuron activations of penultimate layer for CIFAR10.2 for further experiments
from torch.utils.data import TensorDataset
import numpy as np

test_data = np.load('cifar102_train.npz')

test_data_2 = np.transpose(test_data['images'], (0, 3, 1, 2))/255. #Required before the images can be fed to the model

test_data_2 = TensorDataset(torch.FloatTensor(test_data_2), torch.LongTensor(test_data['labels']))
testloader_2 = torch.utils.data.DataLoader(test_data_2,
    batch_size=args['batch_size'], shuffle=False, **kwargs)

all_activs_2, all_outputs_2, all_images_2, all_cats_2 = act_extract(testloader_2, models)

# Step 2. Save Neural Activity for the App

In [ ]:
!pip install deephys==0.7.2
from deephys import import_test_data, Neuron, Layer, Model
import numpy as np

## **Declare the model**

In [11]:
# Declare penultimate layer
neuronList = []
for i in range(np.shape(all_activs)[1]):
  neuronList.append(Neuron())

layerList = []
layerList.append(Layer(
    layerID = "linear1",
    neurons = neuronList
))

# Declare output layer
neuronList = []
for i in range(np.shape(all_outputs)[1]):
  neuronList.append(Neuron())

layerList.append(Layer(
    layerID = "classification",
    neurons = neuronList
))


# Deephys model
model = Model(
    name = "resnet18_cifar",
    suffix = None,
    layers = layerList
)

model.save()

## **Save data for deephys**

In [12]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Save CIFAR10
test = import_test_data(
    "CIFAR10",
    classes,  #List with all category names
    [all_activs,all_outputs], #List with neural activity
    model, #Structure describing the model (see documentation)
    all_images, #Images resized to 32x32 pixels
    all_cats.numpy().tolist() #Labels
    )
test.suffix = None
test.save()

# Save CIFAR10v2
test = import_test_data(
    "CIFARV2",
    classes,
    [all_activs_2,all_outputs_2],
    model,
    all_images_2,
    all_cats_2.numpy().tolist()
    )
test.suffix = None
test.save()